In [49]:
import os
import numpy as np 
import re
import io
#from utilities.features_csv import to_csv
import pandas as pd
from IPython.display import display
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import word_tokenize

In [50]:
import re 
import csv 
docs = {}
labels = {}
#I only work if you run me in the same folder as the actual text files 

with open('training_labels.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            line_count += 1
        else:
            id = int(row[1])
            score= float(row[2])
            labels[id]=score
            line_count += 1

for root, dirs, files in os.walk("corpus/"):
    for file in files:
        if file.endswith(".txt"):
            id = re.sub("[^0-9]", "", file)
            id = int(id) 
            path_file = os.path.join(root,file)
            curdir = path_file
            file = open(curdir, 'r', errors='ignore')
            text = file.read()
            docs[id] = text

        

In [51]:
#auto features here 
import os 
import numpy as np 
import os
import re 
import csv 
import math 
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import word_tokenize
import pandas as pd 
train_x,train_y = [],[]
for k in labels:
    try:
        if docs[k] and labels[k]:
            train_x.append(docs[k]) 
            train_y.append(labels[k])
    except KeyError:
        pass
from sklearn.feature_extraction.text import CountVectorizer

def vectorize(txt):
    count_vect = CountVectorizer(stop_words=stopwords.words('english'),max_features = 6,strip_accents ='ascii')
    #print(count_vect.get_feature_names)
    vects = count_vect.fit_transform(txt)
    print(vects.shape) 
    return count_vect

from sklearn.preprocessing import normalize 
def normalize_vect(v):
    return normalize(v)
    
v = vectorize(train_x)
imp_words = v.get_feature_names()
id_ct = {}
for id in docs:
    id_ct[id] = [0]*len(imp_words)
for id in id_ct:
    txt = docs[id] 
    for x in range(len(imp_words)):
        word = imp_words[x]
        if word in txt:
            id_ct[id][x] +=1
dat = pd.DataFrame(data = id_ct)
dat = dat.transpose()
dat.columns = imp_words
dat['iD'] = dat.index
dat.head()

(434, 6)


,data,information,may,personal,privacy,use,iD
10205,1,1,1,1,1,1,10205
2534,1,1,1,1,1,1,2534
10946,1,1,1,1,1,1,10946
392,0,1,1,0,1,1,392
10173,1,1,1,1,1,1,10173


In [52]:
# dat.head()

In [53]:
data = pd.read_csv("testing.csv")
frames = [data,dat]
data =  data.merge(dat,how='left',on='iD')
display(data.head(n=10))

,iD,minor,geo-location,contact_email,vendors,sell_personal,share_personal,Cookies,fog_index,avg_sentence_length,flesch_reading_ease,dale_chall_readability_score,Score,data,information,may,personal,privacy,use
0,20481,1,0,0,1,0,0,0,75.245659,183.114146,-4.42,9.08,3.0,1,1,1,1,1,1
1,20484,0,0,1,0,0,1,1,58.618557,141.546392,37.79,7.02,3.0,1,1,1,1,1,1
2,854,0,0,0,0,0,0,0,55.264865,133.162162,46.30,6.60,2.0,1,1,1,1,1,1
3,20490,1,0,1,0,0,0,1,70.405195,171.012987,7.88,8.48,3.0,1,1,1,1,1,1
4,20493,0,0,0,0,0,0,0,81.172242,197.930605,-10.99,9.82,2.0,1,1,1,0,1,1
5,173,0,0,1,0,0,0,1,47.731507,114.328767,65.41,5.67,3.0,1,1,1,1,1,1
6,11096,0,0,1,0,0,0,0,49.727168,119.317919,60.35,5.92,2.0,1,1,1,1,1,1
7,20500,0,0,0,0,0,0,1,92.804233,227.010582,-48.97,11.26,2.0,1,1,1,1,1,1
8,516,0,1,0,0,0,0,1,66.037158,160.092896,18.96,7.94,2.0,1,1,1,1,1,1
9,20506,0,0,0,0,0,0,0,42.903529,102.258824,77.66,5.07,2.0,0,1,1,1,1,1


In [54]:
#spliting labels and features
scores = data['Score']
features_raw = data.drop(['Score', 'iD'] , axis = 1)
features_raw = features_raw.drop(["sell_personal",'geo-location'],axis=1)
# features_raw = features_raw.drop(['geo-location'],axis=1)
features_raw = features_raw.drop(['data','information','use','privacy','personal'],axis=1)
features_raw.head()

,minor,contact_email,vendors,share_personal,Cookies,fog_index,avg_sentence_length,flesch_reading_ease,dale_chall_readability_score,may
0,1,0,1,0,0,75.245659,183.114146,-4.42,9.08,1
1,0,1,0,1,1,58.618557,141.546392,37.79,7.02,1
2,0,0,0,0,0,55.264865,133.162162,46.30,6.60,1
3,1,1,0,0,1,70.405195,171.012987,7.88,8.48,1
4,0,0,0,0,0,81.172242,197.930605,-10.99,9.82,1


In [55]:
from sklearn.preprocessing import MinMaxScaler
# Initialize a scaler, then apply it to the features
scaler = MinMaxScaler()
numerical = ['fog_index', 'avg_sentence_length', 'flesch_reading_ease', 'dale_chall_readability_score']
features_log_minmax_transform = pd.DataFrame(data = features_raw)
features_log_minmax_transform[numerical] = scaler.fit_transform(features_raw[numerical])


# Show an example of a record with scaling applied
display(features_log_minmax_transform.head(n = 6))

,minor,contact_email,vendors,share_personal,Cookies,fog_index,avg_sentence_length,flesch_reading_ease,dale_chall_readability_score,may
0,1,0,1,0,0,0.660803,0.660803,0.332320,0.660400,1
1,0,1,0,1,1,0.418325,0.418325,0.569989,0.418331,1
2,0,0,0,0,0,0.369417,0.369417,0.617905,0.368978,1
3,1,1,0,0,1,0.590213,0.590213,0.401577,0.589894,1
4,0,0,0,0,0,0.747232,0.747232,0.295327,0.747356,1
5,0,1,0,0,1,0.259556,0.259556,0.725507,0.259694,1


In [119]:
# Import train_test_split
from sklearn.model_selection import train_test_split
# Split the 'features' and 'income' data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_log_minmax_transform, 
                                                    scores, 
                                                    test_size = 0.03, 
                                                    random_state = 0)
# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 420 samples.
Testing set has 14 samples.


In [120]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=15, max_depth=31,random_state=1, criterion="entropy", max_features = 3)
forest.fit(X_train, y_train)

predictions = forest.predict(X_test)
from sklearn.metrics import f1_score
f1_score(y_test, predictions, average='macro')

0.8531746031746031

In [122]:
from sklearn.tree import DecisionTreeClassifier
dt  = DecisionTreeClassifier(random_state = 47, criterion='gini', splitter = 'random', max_depth=14, max_features = 3)
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=14,
            max_features=3, max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=47, splitter='random')

In [124]:
# dt.fit(X_train, y_train)
predictions = dt.predict(X_test)
from sklearn.metrics import f1_score 
f1_score(y_test, predictions, average = 'macro')

0.8589743589743589

In [130]:
from sklearn.ensemble import ExtraTreesClassifier


ET = ExtraTreesClassifier(n_estimators = 35, random_state = 0, criterion='entropy')
ET.fit(X_train, y_train)
predi = ET.predict(X_test)
ans = f1_score(y_test, predictions, average = 'macro')
print(ans)

0.8589743589743589


In [160]:
from sklearn import model_selection
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import ExtraTreesClassifier
X = features_log_minmax_transform
y = scores
num_folds = 5
num_instances = len(X)
skf = StratifiedKFold(n_splits=10, shuffle=True)
model = dt
results = model_selection.cross_val_score(model, X, y, cv=skf)
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))


Accuracy: 79.047% (4.500%)


In [116]:
from sklearn.metrics import classification_report
#target_names = ['1', '2', '3', '4', '5']
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         1.0       0.59      0.90      0.72        21
         2.0       0.89      0.74      0.81        65
         3.0       0.84      0.76      0.80        34
         4.0       0.60      0.75      0.67         8
         5.0       0.75      1.00      0.86         3

   micro avg       0.78      0.78      0.78       131
   macro avg       0.73      0.83      0.77       131
weighted avg       0.81      0.78      0.78       131



In [117]:
from sklearn.metrics import confusion_matrix

In [118]:
confusion_matrix(y_test,predictions)

array([[19,  0,  2,  0,  0],
       [12, 48,  3,  2,  0],
       [ 1,  4, 26,  2,  1],
       [ 0,  2,  0,  6,  0],
       [ 0,  0,  0,  0,  3]])

In [85]:
# del train_x 
# del id_ct
# del docs 
# del labels 
# del data 
# del dat 
##clear old vars

import os 
import numpy as np 
import os
import re 
import csv 
import math 
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import word_tokenize
import pandas as pd 
import re 
import csv 

docs = {}

for root, dirs, files in os.walk("corpus/testing_set"):
    for file in files:
        if file.endswith(".txt"):
            id = re.sub("[^0-9]", "", file)
            id = int(id) 
            #print(id)
            path_file = os.path.join(root,file)
            curdir = path_file
            file = open(curdir, 'r')
            text = file.read()
            docs[id] = text

#mind you the test data set does not come with the labels...
train_x= []

for k in docs:
    try:
        if docs[k]:
            train_x.append(docs[k]) 
    except KeyError:
        pass
    
#get vectoized features 

from sklearn.feature_extraction.text import CountVectorizer

def vectorize(txt):
    count_vect = CountVectorizer(stop_words=stopwords.words('english'),max_features = 6,strip_accents ='ascii')
    #print(count_vect.get_feature_names)
    vects = count_vect.fit_transform(txt)
    print(vects.shape) 
    return count_vect

v = vectorize(train_x)
imp_words = v.get_feature_names()
id_ct = {}
for id in docs:
    id_ct[id] = [0]*len(imp_words)
for id in id_ct:
    txt = docs[id] 
    for x in range(len(imp_words)):
        word = imp_words[x]
        if word in txt:
            id_ct[id][x] +=1
            
vectorized = pd.DataFrame(data = id_ct)
vectorized = vectorized.transpose()
vectorized.columns = imp_words
vectorized['iD'] = vectorized.index
data = pd.read_csv("testing_data_set.csv")
data =  vectorized.merge(data,how='left',on='iD')
display(final.head(n=5))

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [86]:
allids = data['iD']
features_raw = data.drop(['iD'] , axis = 1)
features_raw = features_raw.drop(["sell_personal",'geo-location'],axis=1)
# features_raw = features_raw.drop(['geo-location'],axis=1)
features_raw = features_raw.drop(['data','information',"use",'privacy','personal'],axis=1)
features_raw.head()

,minor,contact_email,vendors,share_personal,Cookies,fog_index,avg_sentence_length,flesch_reading_ease,dale_chall_readability_score,Score,may
0,1,0,1,0,0,75.245659,183.114146,-4.42,9.08,3.0,1
1,0,1,0,1,1,58.618557,141.546392,37.79,7.02,3.0,1
2,0,0,0,0,0,55.264865,133.162162,46.30,6.60,2.0,1
3,1,1,0,0,1,70.405195,171.012987,7.88,8.48,3.0,1
4,0,0,0,0,0,81.172242,197.930605,-10.99,9.82,2.0,1


In [87]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
numerical = ['fog_index', 'avg_sentence_length', 'flesch_reading_ease', 'dale_chall_readability_score']
features_log_minmax_transform = pd.DataFrame(data = features_raw)
features_log_minmax_transform[numerical] = scaler.fit_transform(features_raw[numerical])
alldata = features_log_minmax_transform
preds = ET.predict(alldata)

ValueError: Number of features of the model must match the input. Model n_features is 10 and input n_features is 11 

## GENERATE PREDICTIONS FOR TEST SET 

In [182]:
answer = pd.DataFrame(allids)

NameError: name 'allids' is not defined

In [183]:
answer["score"] = preds

NameError: name 'preds' is not defined

In [184]:
answer.columns = ["id","score"]
answer.head()

NameError: name 'answer' is not defined

In [91]:
file = open("base_predicions.csv","w") 
colsnrows = answer.to_csv(index = False)
file.write(colsnrows)
file.close()

In [162]:
data = pd.read_csv("final.csv")
frames = [data,dat]
data =  data.merge(dat,how='left',on='iD')
display(data.head(n=10))

scores = data['Score']
features_raw = data.drop(['Score', 'iD'] , axis = 1)
features_raw = features_raw.drop(["sell_personal",'geo-location'],axis=1)
# features_raw = features_raw.drop(['geo-location'],axis=1)
features_raw = features_raw.drop(['data','information','use','privacy','personal'],axis=1)
features_raw.head()

,iD,minor,geo-location,contact_email,vendors,sell_personal,share_personal,Cookies,fog_index,avg_sentence_length,flesch_reading_ease,dale_chall_readability_score,Score,data,information,may,personal,privacy,use
0,20481,1,0,0,1,0,0,0,75.245659,183.114146,-4.42,9.08,3.0,1,1,1,1,1,1
1,20484,0,0,1,0,0,1,1,58.618557,141.546392,37.79,7.02,3.0,1,1,1,1,1,1
2,854,0,0,0,0,0,0,0,55.264865,133.162162,46.30,6.60,2.0,1,1,1,1,1,1
3,20490,1,0,1,0,0,0,1,70.405195,171.012987,7.88,8.48,3.0,1,1,1,1,1,1
4,20493,0,0,0,0,0,0,0,81.172242,197.930605,-10.99,9.82,2.0,1,1,1,0,1,1
5,173,0,0,1,0,0,0,1,47.731507,114.328767,65.41,5.67,3.0,1,1,1,1,1,1
6,11096,0,0,1,0,0,0,0,49.727168,119.317919,60.35,5.92,2.0,1,1,1,1,1,1
7,20500,0,0,0,0,0,0,1,92.804233,227.010582,-48.97,11.26,2.0,1,1,1,1,1,1
8,516,0,1,0,0,0,0,1,66.037158,160.092896,18.96,7.94,2.0,1,1,1,1,1,1
9,20506,0,0,0,0,0,0,0,42.903529,102.258824,77.66,5.07,2.0,0,1,1,1,1,1


,minor,contact_email,vendors,share_personal,Cookies,fog_index,avg_sentence_length,flesch_reading_ease,dale_chall_readability_score,may
0,1,0,1,0,0,75.245659,183.114146,-4.42,9.08,1
1,0,1,0,1,1,58.618557,141.546392,37.79,7.02,1
2,0,0,0,0,0,55.264865,133.162162,46.30,6.60,1
3,1,1,0,0,1,70.405195,171.012987,7.88,8.48,1
4,0,0,0,0,0,81.172242,197.930605,-10.99,9.82,1


In [185]:
print(ET.predict(features_raw.iloc[[5]]))

[2.]


In [176]:
features_raw.iloc[[0]]

,minor,contact_email,vendors,share_personal,Cookies,fog_index,avg_sentence_length,flesch_reading_ease,dale_chall_readability_score,may
0,1,0,1,0,0,75.245659,183.114146,-4.42,9.08,1
